<a href="https://colab.research.google.com/github/VigneshwaraChinnadurai/DL-Learning/blob/master/Part_3_Recurrent_Neural_Networking/RNN_dl_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Recurrent Neural Network

# In this session, we not only predict based on the input, we predict based on the input and previous predicted values.
# This is helpful especially in framing sentences. when it involves gender and tenses.
# This is like google search engine prediction of our search character by character.

# In this dataset, we are going to predict using the previous stock input values to detect the present stock value.


In [0]:
# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# When ever you have any doubt in this networking, google keras documentaton and read that. It will help you.

# Importing the training set
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
# Here we are using Normalisation method(ie minmaxscaler)
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
# This NN is used to predict the words while using google search engine.

# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 1258):
    # Here we have 60 time steps that is checking -60 days the data before predicting
    # 1258 is the upper bound of the dataset
    # So reating the training data like this. If you feel unclear about this refer step 4 in tutorial
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# When we want to include another dimention for better prediction, use this reshape to add a dimention in array.
# Keras is s=expecting a 3d array to work on it.
# 3rd dimention is predictors ie the indicator.
# X_train.shape[0] gives the no of rows.
# X_train.shape[1] gives the no of columns.
# We are giving 1 as 3rd argument as we are having only one indicator.
# If we have another indicator such as google's netflow(as we are dealing with google stock price) add that in place of 1.



In [0]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
# Here we will add only 2nd and 3rd dimention in input shape as the 1st dimention is automatically taken by the function.
# Here units can be reminded as neurons and to have a better prediction, we give that value to be 50 
# Since we are going to add another layer, keeping the return_sequence as true.
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
# Since we are not going to add another layer, keeping the return_sequence as false which is default value.
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# Try using RMS optimizer.
# Universal optimizer is adam.

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
# Update the weights every 32nd time. (Batch_size)


In [0]:
# Part 3 - Making the predictions and visualising the results

# Getting the real stock price of 2017
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

# Getting the predicted stock price of 2017
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
# As we need previous 60 values to predict the current data, we are concatinating the training set and the test set.
# Axis is 0 as it represents we need to concatinate on rows.
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
# To get the values as required for test dataset.
inputs = inputs.reshape(-1,1)
# Just reshaping like 4x1 to 1x4
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
# To inverse or to return true value from the scaled value

# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()